# Test the classical SRNN

This is a notebook for testing the classical SRNN.

## Import everything

Modify setting for pytorch

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
currentPath=os.getcwd()

Import matplotlib and others

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
from torch import nn

Import the classical SRNN and others

In [3]:
#Modify path for the notebooks
currentPath=os.path.join(currentPath,'..')
currentPath=os.path.join(currentPath,'src')
os.chdir(currentPath)

In [4]:
from DataGenerator.HenonMapDataGen import HenonMapDataGen
from ClassicalModels.ClassicalSRNNs import ClassicalSRNN
from ClassicalModels.ClassicalSRNNs import SuportFunction
from GradientFreeOptimizers.CostFunc import GradFreeMSELoss
import GradientFreeOptimizers.Helpers as hp

## Test One

### Get the data

Set save path

In [5]:
savepath=os.path.join(currentPath,'..\data\HenonMap\Test')
filename='HenonMapTest1.csv'

Read the data

In [6]:
hmap=HenonMapDataGen(savepath=savepath)
hmap.read_from_CSV(filename)

In [7]:
print(hmap)

Data Info:
----------------------------------------
Data Size: 1000
Data Interval: 1
Data ParamA: 1.4, Data ParamB: 0.3
Data Bound: -1.2
Data HeavyMem: True
Data Seed:
 [0.02825174685561237, 0.08827631523818379]
----------------------------------------


Generate the data iter

In [8]:
testSetRatio=0.2
numStep=5
batchSize=4

In [9]:
trainIter,testIter=hmap.get_data_iter(testSetRatio,numStep,batchSize,mask=0,shuffle=False)

In [10]:
X,Y=next(iter(trainIter))
print('Train Data Size:',len(trainIter))
print('X=',torch.squeeze(X))
print('Y=',torch.squeeze(Y))

Train Data Size: 156
X= tensor([[ 0.0283,  0.0883,  0.9976, -0.3667,  1.1110],
        [-0.2932,  0.5535,  0.4831,  0.8393,  0.1588],
        [-0.5704,  0.2684,  0.7280,  0.3385,  1.0580],
        [ 1.0608, -0.4793,  0.9967, -0.5344,  0.8991]])
Y= tensor([[ 0.0000,  0.9976, -0.3667,  1.1110, -0.8381],
        [ 0.5535,  0.4831,  0.8393,  0.1588,  1.2165],
        [ 0.2684,  0.7280,  0.3385,  1.0580, -0.4657],
        [-0.4793,  0.9967, -0.5344,  0.8991, -0.2921]])


In [11]:
X,Y=next(iter(testIter))
print('Train Data Size:',len(testIter))
print('X=',torch.squeeze(X))
print('Y=',torch.squeeze(X))

Train Data Size: 36
X= tensor([[ 0.6972,  0.4958,  0.8650,  0.1012,  1.2451],
        [ 1.1484, -0.7807,  0.4913,  0.4279,  0.8911],
        [ 0.3035,  1.1060, -0.6214,  0.7912, -0.0627],
        [-0.5884,  0.8441, -0.1741,  1.2108, -1.1047]])
Y= tensor([[ 0.6972,  0.4958,  0.8650,  0.1012,  1.2451],
        [ 1.1484, -0.7807,  0.4913,  0.4279,  0.8911],
        [ 0.3035,  1.1060, -0.6214,  0.7912, -0.0627],
        [-0.5884,  0.8441, -0.1741,  1.2108, -1.1047]])


### Define the SRNN

#### Get neccesary functions

In [12]:
srnnTest1Sup=SuportFunction()

In [13]:
transform=lambda Xs:[torch.squeeze(x) for x in Xs]

In [14]:
init_rnn_state=srnnTest1Sup.get_init_state_fun()
get_params=srnnTest1Sup.get_get_params_fun()
rnn=srnnTest1Sup.get_forward_fn_fun()
predict_fun=srnnTest1Sup.get_predict_fun(outputTransoform=transform)

#### Create the SRNN

In [15]:
inputSize=outputSize=1
hiddenSize=6

In [16]:
net=ClassicalSRNN(inputSize,hiddenSize,outputSize,get_params,init_rnn_state,rnn)

#### Test prediction

In [17]:
state=net.begin_state(batchSize)
Y,newState=net(X,state)
Y.shape, len(newState), newState[0].shape

(torch.Size([20, 1]), 1, torch.Size([4, 6]))

In [18]:
preX,preY=hmap.data_as_tensor
preX,preY=torch.unsqueeze(preX[:5],-1),torch.unsqueeze(preY[:10],-1)
print('preX=',preX)
preY=[y for y in torch.cat((preX[:2],preY[1:]),dim=0)]
print('preY=',preY)

preX= tensor([[ 0.0283],
        [ 0.0883],
        [ 0.9976],
        [-0.3667],
        [ 1.1110]])
preY= [tensor([0.0283]), tensor([0.0883]), tensor([0.9976]), tensor([-0.3667]), tensor([1.1110]), tensor([-0.8381]), tensor([0.3500]), tensor([0.5771]), tensor([0.6387]), tensor([0.6019]), tensor([0.6844])]


In [19]:
preX=torch.unsqueeze(preX,-1)
YHat=predict_fun(preX,net,numPreds=5)
print('YHat=',YHat)

YHat= [tensor(0.0283), tensor(0.0883), tensor(0.9976), tensor(-0.3667), tensor(1.1110), tensor(-0.0001), tensor(-3.6559e-06), tensor(3.7026e-08), tensor(-9.4360e-10), tensor(9.4382e-12)]


#### Train the network

In [20]:
def train(net, train_iter, testIter,updater, num_epochs, loss,
              use_random_iter=False):
    """train for an epoch"""
    animator = hp.Animator(xlabel='epoch', ylabel='Loss',
                            legend=['train','test'], xlim=[10, num_epochs])
    # prediction
    predict = lambda prefix: predict_fun(prefix,net, numPreds=5)
    # train and predict
    for epoch in range(num_epochs):
        trainLoss, speed = SuportFunction.train_epoch(
            net, train_iter, loss, updater, use_random_iter)
        if (epoch + 1) % 10 == 0:
            print(predict(preX))
            testLoss=SuportFunction.evaluate_accuracy(net, testIter, loss, use_random_iter)
            animator.add(epoch + 1, [trainLoss,testLoss])
    testLoss=SuportFunction.evaluate_accuracy(net, testIter, loss, use_random_iter)
    print(f'testLoss {testLoss:.1f}, {speed:.1f} point/s')
    print(predict(preX))